# Overview
this notebook is an implementation of the PSO approach to eliminate multi-collinearily in any given dataset.

In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def display_heat_map(data: pd.DataFrame, title: str):
    plt.figure(figsize=(10,10))
    sns.heatmap(data, linewidth = 1 , annot = True)
    plt.title(title)
    plt.show()

In [3]:
from scipy.stats import spearmanr
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sko.PSO import PSO
from math import floor
from sklearn.decomposition import PCA


class PsoMultiCol:
    def set_data(self, df):
        self.df = df
        # calculate the spearmanr correlation of the dataframe's features
        corr = spearmanr(df).correlation
        # make sure it is symmetric
        corr = (corr + corr.T) / 2
        # fill the diagonal with 1s
        np.fill_diagonal(corr, 1)
        # transform the matrix to a dataframe that represents how similar each feature it is to another
        self.dis_matrix = pd.DataFrame(data= (1 - np.abs (corr)), columns=list(df.columns), index=list(df.columns))
        # have a dictionary mapping the column's order to its name
        self.columns_dict = dict(list(zip(range(len(df.columns)), df.columns)))
        # set the number of features for later reference
        self.num_feats = len(df.columns)
        # save the column names for later reference
        self.columns = list(df.columns)
       
    def __init__ (self, df:pd.DataFrame=None, max_iter:int=200, vif_threshold:float=2.5, epsilon:int=0.1,
                  min_fraction=0.40, max_fraction=0.76, step=0.05): # add other parameters to the game
        self.max_iter = max_iter
        self.pso = None
        # the value that determine whether columns are multicollinear or not
        self.vif_threshold = vif_threshold
        # an epsilon value used in the evaluation function
        self.epsilon = epsilon
        self.min_fraction = min_fraction
        self.max_fraction = max_fraction
        self.step = step
        self.pso = None
        if df is not None:
            self.set_data(df)
    
    def _get_vif(self, df=None):
        if df is None:
            df = self.df
        
        vif = pd.DataFrame()
        vif['VIF'] = [variance_inflation_factor(df.values, i) for i in range(len(df.columns))]
        vif['variables'] = df.columns
        return vif.set_index('variables')   
    
    
    def _get_clusters(self, particle: np.array):
        particle_size = len(particle)
        discrete_particle = np.array([int(x) for x in particle])
        cluster_feats = {}
        for i in range(particle_size) :
            # if the value of the cluster is not in the dictinary, initialize the list
            if discrete_particle[i] not in cluster_feats:
                cluster_feats[discrete_particle[i]] = []
            # the cluster_feats will be a map between numbers representing clusters
            # and columns representing 
            cluster_feats[discrete_particle[i]].append(i)
        
        return cluster_feats    
    
    def _cluster_scores(self, cluster_feats: dict):
        cluster_names = {}
        new_order = []
        title = ""
        
        # map each cluster to its column names
        for c, feats in cluster_feats.items():
            cluster_names[c] = [self.columns_dict[i] for i in feats]
            new_order.extend(feats)
            title += f"{len(feats)}-"
                        
        # how dissimilar the clusters are from each other
        inter_cluster_score = 0
        keys = list(cluster_names.keys())
        n_clusters = len(cluster_feats)
        # for i in range(n_clusters):
        #     for j in range(i + 1, n_clusters):
        #         inter_cluster_score += np.nanmean((self.dis_matrix.loc[cluster_names[keys[i]], cluster_names[keys[j]]]))
        
        # inter_cluster_score /= (n_clusters) * (n_clusters - 1)

        # how similar the points are inside a single cluster 
        inner_cluster_score = 0
        for c, names in cluster_feats.items():
            inner_cluster_score += ( 1 + np.exp(self.dis_matrix.iloc[names, names].values.sum())) / np.log(len(names) + np.exp(1))
            # inner_cluster_score += 1 + np.nanmean(self.dis_matrix.loc[names, names])         
        
        # new_dist_matrix = self.dis_matrix.loc[new_order[::-1], new_order]
        # display_heat_map(new_dist_matrix, title)
        return inner_cluster_score # / inter_cluster_score

    def _pso_function(self, particle: np.array):         
        return self._cluster_scores(self._get_clusters(particle))
    
    def _cluster_pso(self, num_clusters):
        # determine the function object to pass to the PSO algorithm
        pso_function = lambda x: self._pso_function(x)
        # bounds
        lower_bound = np.zeros(self.num_feats)
        upper_bound = np.full(shape=self.num_feats, fill_value = num_clusters, dtype="float")
        
        pso =  PSO(func=pso_function, n_dim = self.num_feats, pop=15, max_iter=self.max_iter ,lb=lower_bound, ub=upper_bound, c1=1.5, c2=1.5)
        pso.run()
        
        x, y = pso.gbest_x, pso.gbest_y
        
        cluster_feats = self._get_clusters(x)
        
        new_order = [] 
        title = ""
        for _, f in cluster_feats.items():
            new_order.extend(f)
            title += f'{len(f)}-'
        
        # print(y)
        # new_dist_matrix = self.dis_matrix.loc[new_order[::-1], new_order]
        # display_heat_map(new_dist_matrix, title)
        
        return x, y
    
    def _find_best_cluster(self):
        best_score = np.inf
        best_x = None   
        last_num_clusters = 0
        for fraction in np.arange(self.min_fraction, self.max_fraction, self.step):            
            num_clusters = max(floor(fraction  * self.num_feats), 3)

            if num_clusters == last_num_clusters: 
                continue
            
            last_num_clusters = num_clusters
            
            x, y = self._cluster_pso(num_clusters)
            if y < best_score:
                best_score = y
                best_x = x
        
        return best_x
    
    def _get_new_df(self, best_particle):
        # define a PCA object to combine the clustered 
        pca = PCA(n_components=1)

        # get the clusters out of the particle
        clusters = self._get_clusters(best_particle)
        # get the cluster
        new_dfs = []
        
        for _, feats in clusters.items():
            # reduce the clustered features into a single more informative feature
            new_feats = pd.DataFrame(data=pca.fit_transform(self.df.iloc[:, feats]), index=list(self.df.index))
            new_dfs.append(new_feats)
        
        # return the features concatenated horizontally 
        return pd.concat(new_dfs, axis=1, ignore_index=True)
                               
    def eliminate_multicol(self, df: pd.DataFrame):
        # first of all determine the vifs of the different columns
        vif = self._get_vif(df)
        # retrieve multicollinear variables
        collinear = list(vif[vif['VIF'] >= self.vif_threshold].index)
        collinear_df = df.loc[:, collinear]

        # retrieve the non-collinear part
        non_collinear = [c for c in df.columns if c not in collinear]
        non_collinear_df = df.loc[:, non_collinear]
        
        # if there are no collinear columns, no further preprocessing is needed
        if not collinear:
            return df
        
        # set the df field to the fraction of the dataframe with only multicollinear columns
        self.set_data(collinear_df)
        # retrieve the best particle
        best_x = self._find_best_cluster()
        # retrieve the new representation of the collinear features
        new_collinear_df = self._get_new_df(best_x)
        # concatenate the two parts to form the final dataframe
        return pd.concat([non_collinear_df, new_collinear_df], axis=1).rename(columns=lambda x: str(x))


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


In [5]:
dataset = pd.read_excel(os.path.join('databases', 'final_dataset.xlsx'), usecols=lambda x: 'Unnamed' not in x)
dataset.drop(columns=['name', 'code'], inplace=True)
y = dataset.pop('pop_growth')

In [6]:
def prepare_train_test(dataset, y, test_size=0.2, random_state=11):
    X_train, X_test, y_train, y_test = train_test_split(dataset, y, test_size=test_size, random_state=random_state)
    X_train.reset_index(inplace=True, drop=True)
    X_test.reset_index(inplace=True, drop=True)

    # for non year columns
    standard_scaler = StandardScaler()
    # for the year column
    mm_scaler = MinMaxScaler()

    X_train['year'] = mm_scaler.fit_transform(X_train.loc[:, ['year']]).reshape(-1, )
    scaled_year = X_train.pop('year')

    X_train = pd.DataFrame(data=standard_scaler.fit_transform(X_train), columns=X_train.columns)
    X_train['year'] = scaled_year

    X_test['year'] = mm_scaler.fit_transform(X_test.loc[:, ['year']]).reshape(-1, )
    scaled_year = X_test.pop('year')

    X_test = pd.DataFrame(data=standard_scaler.fit_transform(X_test), columns=X_test.columns)
    X_test['year'] = scaled_year

    return X_train, X_test, y_train, y_test


In [7]:
dataset.reset_index(drop=True, inplace=True)
X_train, X_test, y_train, y_test = prepare_train_test(dataset, y)

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

ridge_basic = Ridge(max_iter=5000, random_state=11)
ridge_grid = {"alpha": np.logspace(-8, 8, 100)}

lasso_basic = Lasso(max_iter=5000, random_state=11)
lasso_grid = {"alpha": (np.logspace(-8, 8, 100)), }

knn_basic = KNeighborsRegressor()
knn_grid = {'n_neighbors' : list(range(3, 21)),
            'weights' : ['uniform','distance'],
            'metric' : ['minkowski','euclidean','manhattan']
}

dtr_basic = DecisionTreeRegressor(random_state=11)
dtr_grid = {"max_features": np.arange(0.7, 1.01, 0.1), 
            "max_depth": range(10, 21, 2), 
             "min_samples_split":np.arange(0.02, 0.055, 0.01), 
             "min_samples_leaf": range(1, 21, 2)  
}

rfr_basic = RandomForestRegressor(random_state=11)
rfr_grid = {"n_estimators": [100], 
             "max_depth": range(10, 21, 2), 
             "min_samples_split": np.arange(0.02, 0.041, 0.01), 
             "min_samples_leaf": range(1, 7, 2)
}

In [9]:
from regressors import apply_model

ridge_res1 = apply_model(ridge_basic, X_train, X_test, y_train, y_test, ridge_grid, save=True, save_path=os.path.join("ridge_1.pkl"))
print(ridge_res1)

lasso_res1 = apply_model(lasso_basic, X_train, X_test, y_train, y_test, lasso_grid, save=True, save_path=os.path.join("lasso_1.pkl"))
print(lasso_res1)

dtr_res1 = apply_model(dtr_basic, X_train, X_test, y_train, y_test, dtr_grid, save=True, save_path=os.path.join("decision_tree_1.pkl"))
print(dtr_res1)

knn_res1 = apply_model(knn_basic, X_train, X_test, y_train, y_test, knn_grid, save=True, save_path=os.path.join("knn_1.pkl"))
print(knn_res1)

rfr_res1 = apply_model(rfr_basic, X_train, X_test, y_train, y_test, rfr_grid, save=True, save_path=os.path.join("random_forest_1.pkl"))
print(rfr_res1)

(Ridge(alpha=0.18738174228603868, max_iter=5000, random_state=11), {'mse': 0.17924487145299786})
(Lasso(alpha=0.00048626015800653534, max_iter=5000, random_state=11), {'mse': 0.17895109074496957})
(DecisionTreeRegressor(max_depth=20, max_features=0.9999999999999999,
                      min_samples_leaf=3, min_samples_split=0.02,
                      random_state=11), {'mse': 0.6761465793373358})
(KNeighborsRegressor(metric='manhattan', n_neighbors=3, weights='distance'), {'mse': 0.1329608998277794})
(RandomForestRegressor(max_depth=20, min_samples_split=0.02, random_state=11), {'mse': 0.4479113576363257})


In [10]:
# apply multicollinearity elimination
pso = PsoMultiCol()

new_dataset = pso.eliminate_multicol(dataset)
new_dataset.reset_index(drop=True, inplace=True)

X_train_new, X_test_new, y_train, y_test = prepare_train_test(new_dataset, y, test_size=0.2, random_state=11)

In [11]:
# Only test after multicollinearity step

ridge_res2 = apply_model(ridge_basic, X_train_new, X_test_new, y_train, y_test, ridge_grid, save=True, save_path=os.path.join("ridge_2.pkl"))
lasso_res2 = apply_model(lasso_basic, X_train_new, X_test_new, y_train, y_test, lasso_grid, save=True, save_path=os.path.join("lasso_2.pkl"))
dtr_res2 = apply_model(dtr_basic, X_train_new, X_test_new, y_train, y_test, dtr_grid, save=True, save_path=os.path.join("decision_tree_2.pkl"))
knn_res2 = apply_model(knn_basic, X_train_new, X_test_new, y_train, y_test, knn_grid, save=True, save_path=os.path.join("knn_2.pkl"))
rfr_res2 = apply_model(rfr_basic, X_train_new, X_test_new, y_train, y_test, rfr_grid, save=True, save_path=os.path.join("random_forest_2.pkl"))

print(f"Ridge status before removal of multicollinearity: {ridge_res1}\nand after: {ridge_res2}\n")
print(f"Lasso status before removal of multicollinearity: {lasso_res1}\nand after: {lasso_res2}\n")
print(f"Decision tree status before removal of multicollinearity: {dtr_res1}\nand after: {dtr_res2}\n")
print(f"KNN status before removal of multicollinearity: {knn_res1}\nand after: {knn_res2}\n")
print(f"Random forest status before removal of multicollinearity: {rfr_res1}\nand after: {rfr_res2}\n")

Ridge status before removal of multicollinearity: (Ridge(alpha=0.18738174228603868, max_iter=5000, random_state=11), {'mse': 0.17924487145299786})
and after: (Ridge(alpha=23.644894126454073, max_iter=5000, random_state=11), {'mse': 1.2033850493835958})

Lasso status before removal of multicollinearity: (Lasso(alpha=0.00048626015800653534, max_iter=5000, random_state=11), {'mse': 0.17895109074496957})
and after: (Lasso(alpha=0.0031257158496882415, max_iter=5000, random_state=11), {'mse': 1.2031264390413594})

Decision tree status before removal of multicollinearity: (DecisionTreeRegressor(max_depth=20, max_features=0.9999999999999999,
                      min_samples_leaf=3, min_samples_split=0.02,
                      random_state=11), {'mse': 0.6761465793373358})
and after: (DecisionTreeRegressor(max_depth=20, max_features=0.8999999999999999,
                      min_samples_leaf=13, min_samples_split=0.02,
                      random_state=11), {'mse': 0.8924558404302655})

KNN s

In [23]:
from niapy.algorithms.basic import CuckooSearch
from evopreprocess.feature_selection import EvoFeatureSelection

from Feature_Transformer import FeatureTransformer

def use_model(model, grid_params, X_train, X_test, y_train, y_test, save=True, save_path=None):
    # use feature selection
    evo = EvoFeatureSelection(evaluator=model, random_seed=11, optimizer=CuckooSearch, n_runs=5, n_folds=3)
    X_new_train = evo.fit_transform(X_train, y_train)
    X_new_test = evo.transform(X_test)
    # use feature tranformer
    transformer = FeatureTransformer()
    X_new_train = transformer.fit_transform(X_new_train, y_train)
    X_new_test = transformer.transform(X_new_test)

    # convert all the column names to the string type
    X_new_train = X_new_train.rename(columns=lambda x: str(x))
    X_new_test = X_new_test.rename(columns=lambda x: str(x))
    
    # train, hypter tune and save the model according to the passed parameters
    return apply_model(model, X_new_train, X_new_test, y_train, y_test, grid_params, save=save, save_path=save_path)[1]

ridge_res2 = use_model(ridge_basic, ridge_grid, X_train_new, X_test_new, y_train, y_test, save=True, save_path=os.path.join("ridge_2.pkl"))
lasso_res2 = use_model(lasso_basic, lasso_grid, X_train_new, X_test_new, y_train, y_test, save=True, save_path=os.path.join("lasso_2.pkl"))
dtr_res2 = use_model(dtr_basic, dtr_grid, X_train_new, X_test_new, y_train, y_test, save=True, save_path=os.path.join("decision_tree_2.pkl"))
knn_res2 = use_model(knn_basic, knn_grid, X_train_new, X_test_new, y_train, y_test, save=True, save_path=os.path.join("knn_2.pkl"))
rfr_res2 = use_model(rfr_basic, rfr_grid, X_train_new, X_test_new, y_train, y_test, save=True, save_path=os.path.join("random_forest_2.pkl"))

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\evopreprocess\feature_selection\EvoFeatureSelection.py:164: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  features = stats.mode(features, axis=1, nan_policy='omit')[0].flatten()
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\evopreprocess\feature_selection\EvoFeatureSelection.py:164: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `

In [ ]:
print(f"Ridge status before preproc: {ridge_res1}\nand after: {ridge_res2}\n")
print(f"Lasso status before preproc: {lasso_res1}\nand after: {lasso_res2}\n")
print(f"Decision tree status before preproc: {dtr_res1}\nand after: {dtr_res2}\n")
print(f"KNN status before preproc: {knn_res1}\nand after: {knn_res2}\n")
print(f"Random forest status before preproc: {rfr_res1}\nand after: {rfr_res2}\n")

Ridge status before preproc: (Ridge(alpha=0.18738174228603868, max_iter=5000, random_state=11), {'mse': 0.17924487145299786})
and after: {'mse': 0.8180149702243408}

Lasso status before preproc: (Lasso(alpha=0.00048626015800653534, max_iter=5000, random_state=11), {'mse': 0.17895109074496957})
and after: {'mse': 1.2636992356511192}

Decision tree status before preproc: (DecisionTreeRegressor(max_depth=20, max_features=0.9999999999999999,
                      min_samples_leaf=3, min_samples_split=0.02,
                      random_state=11), {'mse': 0.6761465793373358})
and after: {'mse': 0.7908379322579535}

KNN status before preproc: (KNeighborsRegressor(metric='manhattan', n_neighbors=3, weights='distance'), {'mse': 0.1329608998277794})
and after: {'mse': 0.29784931168439743}

Random forest status before preproc: (RandomForestRegressor(max_depth=20, min_samples_split=0.02, random_state=11), {'mse': 0.4479113576363257})
and after: {'mse': 0.5798890605248871}

